# Данные по ресторанам

In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
data = {
    'air_stores': pd.read_csv('dataset/air_store_info.csv'),
    
    'cities': pd.read_csv('dataset/cities.csv'),
}

## Обработка ресторанов

In [3]:
stores_id = data['air_stores']['air_store_id']
data['stores'] = pd.concat(
    [pd.DataFrame({
        'air_store_id': stores_id,
        'visit_dow':    [i] * len(stores_id)
    }) for i in range(7)],
    axis=0,
    ignore_index=True,
).reset_index(drop=True)

In [4]:
data['stores'] = pd.merge(data['stores'], data['air_stores'], how='left', on=['air_store_id'])

In [5]:
data['stores']['air_genre_name'] = data['stores']['air_genre_name']
data['stores']['air_area_name']  = data['stores']['air_area_name']

Добавляем информацию по городам

In [6]:
def distance(a, b):
    return sqrt((a['latitude'] - b['latitude']) ** 2 + (a['longitude'] - b['longitude']) ** 2)

for index, city in data['cities'].iterrows():
    data['stores'][city['name']] = data['stores'].apply(lambda row: distance(city, row), axis=1)
    
data['stores']['city'] = data['stores'][data['cities']['name']].idxmin(axis=1)

data['stores'] = data['stores'].drop(data['cities']['name'], axis=1)

In [7]:
print('stores', data['stores'].shape)

data['stores'].head()

stores (5803, 7)


,air_store_id,visit_dow,air_genre_name,air_area_name,latitude,longitude,city
0,air_0f0cdeee6c9bf3d7,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Tokyo
1,air_7cc17a324ae5c7dc,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Tokyo
2,air_fee8dcf4d619598e,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Tokyo
3,air_a17f0778617c76e2,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Tokyo
4,air_83db5aff8f50478e,0,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tokyo


In [8]:
data['stores'].to_csv('result/stores.csv', index=False)